In [1]:
## Loading libraries

import anndata as ad
import seaborn as sns
from matplotlib import pyplot as plt
import scvi
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.stats import median_abs_deviation
import scanpy as sc
import os

Global seed set to 0
/home/usuario/.local/lib/python3.9/site-packages/tqdm-4.66.2-py3.9.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Gathering single-cell sequencing files (only if necessary)
This code just gather the matrix.mtx, the barcodes.tsv (could be cellid.tsv) and the gene.tsv (could be features.tsv) files into one anndata object.

In [2]:
# Dir to the samples
sample_path = '../sc_data_directory'
# Folders between the sample name and the files (if doesn't exist, put '')
middle_path = 'filtered_feature_bc_matrix'

sample_names = []
for foldername in os.listdir(sample_path):
    if os.path.isdir(os.path.join(sample_path, foldername)):
        sample_names.append(foldername)
print(sample_names)
# For each sample, read the directory to a list of samples.
        
sample_list = []

for name in sample_names:
    # Getting anndata (transposed to obs X vars)
    path = f'{sample_path}/{name}/{middle_path}/matrix.mtx.gz'
    sample = sc.read(path, cache=True).T
    
    # Getting obs
    path = f'{sample_path}/{name}/{middle_path}/barcodes.tsv.gz'
    obs = pd.read_csv(path, sep='\t', header=None, index_col=0)
    obs.index.name = 'barcode'
    sample.obs = obs
    
    # Adding metadata (IN MY CASE, THE INFORMATION OF PATIENT AND CONDITION WAS IN THE FILE NAME)
    sample.obs['Patient'] = name
    sample.obs['Condition'] = name[-1].upper()
    
    # Getting vars
    path = f"{sample_path}/{name}/{middle_path}/features.tsv.gz"
    var = pd.read_table(path, sep='\t', header=None, index_col=1)
    var.index.name = 'genes'
    sample.var = var
    sample.var_names_make_unique(join="-")
    
    sample_list.append(sample)

adata = ad.concat(sample_list)
adata.obs_names_make_unique()
del sample_list

['p031n', 'p027n', 'p018n', 'p032n', 'p034n', 'p030t', 'p033t', 'p027t', 'p023t', 'p030n', 'p018t', 'p033n', 'p032t', 'p029n', 'p034t', 'p031t', 'p024t', 'p019t', 'p028n', 'p019n']


/home/usuario/.local/lib/python3.9/site-packages/anndata-0.10.5.post1-py3.9.egg/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


## HVG

In [4]:
adata.layers['counts']=adata.X.copy()
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=5000, 
                            layer='counts',subset=True, batch_key='Patient')

## Doublet removal with SOLO (SCVi)

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer='counts')
model = scvi.model.SCVI(adata)
model.train(max_epochs=100)

solo = scvi.external.SOLO.from_scvi_model(model)
solo.train()

solo_df = solo.predict()
solo_df
cell_mapper = dict(zip(solo_df.index, solo_df.prediction))
adata.obs['doublet_pred'] = adata.obs.index.map(cell_dict)

/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float3

Epoch 1/100:   0%|          | 0/100 [00:00<?, ?it/s]

/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


Epoch 95/100:  94%|█████████▍| 94/100 [05:14<00:19,  3.30s/it, loss=1.15e+03, v_num=1]

/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


INFO     Creating doublets, preparing SOLO model.                                                                  


In [ ]:
adata[~adata.obs.doublet_pred].shape()

## QC

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')
adata.var['ribo'] = adata.var_names.str.startswith(('RPS','RPL'))
adata.var['hb'] = adata.var_names.str.startswith(("^HB[^(P)]"))

sc.pp.calculate_qc_metrics(
    adata, qc_vars=['mt','ribo','hb'], inplace=True, percent_top=[20], log1p=True
)

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_ribo',
                     'pct_counts_hb', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)

adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)

adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

In [3]:
adata = ad.read_h5ad("../h5ad_Datasets/atlas_v2_raw.h5ad")

In [4]:
print(adata.X[0:10,0:10])

  (0, 8)	2.0
  (0, 5)	1.0
  (1, 3)	1.0
  (2, 7)	2.0
  (3, 7)	2.0
  (3, 3)	2.0
  (4, 7)	1.0
  (5, 8)	2.0
  (6, 7)	1.0
  (8, 3)	1.0


## Normalize

In [5]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
#adata.raw = adata

#sc.pl.highest_expr_genes(adata, n_top=20)

## scVI Integration

In [6]:
scvi.model.SCVI.setup_anndata(adata, batch_key='patient')

arches_params = dict(
    use_layer_norm='both',
    use_batch_norm='none',
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2,
)

vae = scvi.model.SCVI(adata, **arches_params)
vae.train()

adata.obsm["X_scVI"] = vae.get_latent_representation() #dimensional reduction
adata.layers['scvi_normalized'] = vae.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.umap(adata)

/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. 

Epoch 1/16:   0%|          | 0/16 [00:00<?, ?it/s]

/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:476: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(
/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:476: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(


Epoch 2/16:   6%|▋         | 1/16 [00:38<09:36, 38.44s/it, loss=4.54e+03, v_num=1]

/home/usuario/miniconda3/envs/atlas_protocol/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:476: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(


Epoch 16/16: 100%|██████████| 16/16 [10:03<00:00, 37.67s/it, loss=4.44e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=16` reached.


Epoch 16/16: 100%|██████████| 16/16 [10:03<00:00, 37.70s/it, loss=4.44e+03, v_num=1]


In [7]:
adata.write_h5ad("adata_processed_genes18k.h5ad") 